In [89]:
import matplotlib as mpl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from tqdm import tqdm

from decimal import Decimal


**Parameters need for the calculations**

1.The photodiode signal due to Fluorescence (mV)

2.Resistance of Photodiode 
$$I = U/R$$

3.The reponsitivity of the photodiode
$$P = I/responsitivity$$

4.The fluorescence rate per atom/scattering rate (photons/s)

5.The photon flux(Photons/s) 
$$r_p= P/hv$$

Then Calculate how many atoms got into PD. With Solid angle, calculate the total number of atoms in the MOT

6.solid angle: 
$$Omega = A/d^2$$

where the area of the lens surface and d being the distance from the lens to the object.

7.3D MOT Size: 1.373 mm, 1.626 mm

Chip size : 3*3 mm





In [90]:
def error_sum_diff(error1, error2):
    """
    Calculate the error in the sum or difference of two values.
    """
    return math.sqrt(error1**2 + error2**2)

def error_product_quotient(value1, error1, value2, error2, operation='product'):
    """
    Calculate the error in the product or quotient of two values.
    """
    if operation == 'product':
        result = value1 * value2
        return abs(result) * math.sqrt((error1/value1)**2 + (error2/value2)**2)
    elif operation == 'quotient':
        result = value1 / value2
        return abs(result) * math.sqrt((error1/value1)**2 + (error2/value2)**2)
    else:
        raise ValueError("Invalid operation. Use 'product' or 'quotient'.")

In [124]:
P_v = 0.3# V #Photodiode Signal

V_err = 0.01

gain = 2.38 * 10**(6) #V/A 70db

gain_err = gain*0.02


Respons = 0.12

Respons_err = 0.005



c = 299792458 #m/s

h = 6.62607015*10**(-34)# Joule*second

wavelength = 460.862022*10**(-9)

wave_err = 0.000002*10**(-9)

f = c/wavelength# m/s

f_err = error_product_quotient(c,0,wavelength,wave_err, operation='quotient')

E = h*f 

E_err = error_product_quotient(h,0,f,f_err, operation='product')


In [125]:
P_I = P_v/gain #PD Current

I_err = error_product_quotient(P_v,V_err,gain,gain_err, operation='quotient')

In [126]:
print("The current in the photodioe is ""{:.3E}".format(Decimal(P_I)), "+/-", I_err/P_I*100 ,"%", "A")

The current in the photodioe is 1.261E-7 +/- 3.8873012632302 % A


In [127]:
P_P = P_I/Respons # Light incident power
P_err = error_product_quotient(P_I,I_err,Respons,Respons_err, operation='quotient')

In [128]:
print("The incident lignt power is ""{:.3E}".format(Decimal(P_P)), "+/-", P_err/P_P*100,"%", "W")

The incident lignt power is 1.050E-6 +/- 5.698440332426253 % W


In [129]:
r_p=P_P/E # photons per second/photon flux
r_p_err = error_product_quotient(P_P,P_err,E,E_err, operation='quotient')

In [130]:
print('The photon flux is '"{:.3E}".format(Decimal(r_p)), "+/-",r_p_err/r_p*100,"%","photons per second")

The photon flux is 2.437E+12 +/- 5.698440332426269 % photons per second


Scattering rate of the strontinum atoms

\begin{equation}
R = \frac{\Gamma}{2} \frac{I/I_{sat}}{1 + I/I_{sat} + 4\delta^2/\Gamma^2}.
\end{equation}

In [131]:
gamma = 2*np.pi*30.41*10**(6) #The natural linewidth of the transition

I_Isat = 0.9

detune = -60 *10**6 #Hz detuning of the 3D MOT beam
detune_err = 3 * 10**(6)

In [132]:
R = gamma/2*I_Isat/(1+I_Isat+4*detune**2/gamma**2)

R_err = R*detune_err/detune

In [133]:
print('The scattering rate of the strontinum atoms is '"{:.3E}".format(Decimal(R)), "+/-",-R_err/R*100,"%", "photons per second per atom")

The scattering rate of the strontinum atoms is 3.747E+7 +/- 5.0 % photons per second per atom


Atom number at the photodiode

In [134]:
N_pd = r_p/R # the number of atoms into the photodiode

N_pd_err = error_product_quotient(r_p,r_p_err,R,R_err, operation='quotient')

In [135]:
print('The number of atoms into the photodiode is '"{:.3E}".format(Decimal(N_pd)), "+/-",N_pd_err/N_pd*100,"%", "photons per second per atom")

The number of atoms into the photodiode is 6.503E+4 +/- 7.581043610362785 % photons per second per atom


In [136]:
#calculate the solid angle
r = 12.7*10**(-3)/2
r_err = 0.1*10**(-3)/2# mm
r_sqre = (12.7*10**(-3)/2)**2
r_sqre_err = error_product_quotient(r,r_err,r,r_err, operation='product')


A = np.pi*r_sqre
A_err=error_product_quotient(np.pi,0,r_sqre,r_sqre_err, operation='product')

d = 0.15
d_err=0.001
d_sqre = d**2
d_sqre_err = error_product_quotient(d,d_err,d,d_err, operation='product')

Omega = A/d**2
Omega_err = error_product_quotient(A,A_err,d_sqre,d_sqre_err, operation='quotient')

In [137]:
print('The solid angle is '"{:.3E}".format(Decimal(Omega)), "+/-",Omega_err/Omega*100,"%")

The solid angle is 5.630E-3 +/- 1.4590720917397635 %


In [138]:
N_r = N_pd*4*np.pi/Omega
N_r_err = N_r* math.sqrt((N_pd_err/N_pd)**2 + (Omega_err/Omega)**2)

In [139]:
print('The 3D Blue MOT currently have '"{:.3E}".format(Decimal(N_r)),"+/-",N_r_err/N_r*100,"%","atoms")

The 3D Blue MOT currently have 1.452E+8 +/- 7.720175748719472 % atoms
